In [ ]:
import graphistry
import json
graphistry.__version__

### Settings

In [ ]:
PROJECT_ID   = "my_project" 
INSTANCE_ID  = "my_instance" 
DATABASE_ID  = "finance-graph-db"  

# optional setting to limit the number of records returned
LIMIT_CLAUSE = ""
# or use: 
# LIMIT_CLAUSE = "limit 1000"  

SPANNER_CONF = { "project_id":  PROJECT_ID, 
                 "instance_id": INSTANCE_ID, 
                 "database_id": DATABASE_ID }

# print(json.dumps(SPANNER_CONF, indent=4))

### Graphistry register and gcloud init

In [ ]:
# graphistry register 

# To specify Graphistry account & server, use:
# graphistry.register(api=3, username='...', password='...', protocol='https', server='hub.graphistry.com')
# For more options, see https://pygraphistry.readthedocs.io/en/latest/server/register.html

graphistry.register(api=3,  
                    protocol  = "http", 
                    server    = os.getenv("GRAPHISTRY_SERVER"),
                    username  = os.getenv("GRAPHISTRY_USERNAME"),         
                    password  = os.getenv("GRAPHISTRY_PASSWORD"), 
                    spanner_config=SPANNER_CONF
                   )

In [ ]:
# Set the google project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

In [ ]:
!gcloud auth application-default login

### Example 1: GQL Path Query to Graphistry Visualization of all nodes and edges (LIMIT optional) 

to extract the data from Spanner Graph as a graph with nodes and edges in a single object, a GQL path query is required. 

The format of a path query is as follows, note the p= at the start of the MATCH clause, and the SAFE_TO_JSON(p) without these, 
the query will not produce the results needed to properly load a graphistry graph. LIMIT is optional, but for large graphs with millions
        of edges or more, it's best to filter either in the query or use LIMIT so as not to exhaust GPU memory.  

```python
GRAPH FinGraph
MATCH p = (a)-[b]->(c) where 1=1 LIMIT 10000 return SAFE_TO_JSON(p) as path
```


In [ ]:
query=f'''GRAPH FinGraph
MATCH p = (a)-[b]->(c) where 1=1 {LIMIT_CLAUSE} return SAFE_TO_JSON(p) as path'''

g = graphistry.spanner_gql_to_g(query)

In [ ]:
g.plot()

#### Example 1.1 - inspect contents of graphistry graph (nodes and edges): 

In [ ]:
len(g._nodes), len(g._edges)

In [ ]:
g._nodes.head(3)

In [ ]:
g._edges.head(3)

### Example 2: Spanner GQL Query to pandas dataframe (LIMIT optional) 

This example shows a non-path query that returns tabular results, which are then convered to a dataframe for easy manipulation and inspection of the results. 

```python
GRAPH FinGraph 
MATCH (p:Person)-[]-()->(l:Loan)
RETURN p.id as ID, p.name AS Name, SUM(l.loan_amount) AS TotalBorrowed
ORDER BY TotalBorrowed DESC
LIMIT 10```



In [ ]:
query_top10='''GRAPH FinGraph 
MATCH (p:Person)-[]-()->(l:Loan) WHERE 1=1
RETURN p.id as ID, p.name AS Name, SUM(l.loan_amount) AS TotalBorrowed
ORDER BY TotalBorrowed DESC
LIMIT 10'''

In [ ]:
Top10_Borrowers_df = graphistry.spanner_query_to_df(query_top10)

In [ ]:
Top10_Borrowers_df.head(10)